In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string
from string import digits
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df_train = pd.read_csv("../data/train.csv")
# 3 столбца - id, text, author
df_train.head(n=3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [5]:
remove_digits = str.maketrans('', '', digits)
def tokenize_stem(file_text):
    #firstly let's apply nltk tokenization
    file_text = file_text.translate(remove_digits)
    
    tokens = nltk.word_tokenize(file_text)

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #deleting stop_words
    stop_words = stopwords.words('english')
    tokens = [i for i in tokens if ( i not in stop_words )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return tokens

In [6]:
df_train['cleaned_text'] = df_train.text.apply(tokenize_stem)
df_train['cleaned_text_string'] = df_train.cleaned_text.apply(' '.join)
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...


In [7]:
df_train['length']=df_train['cleaned_text_string'].apply(len)
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116


In [8]:
df_hpl=df_train[df_train['author']=='HPL']
df_hpl.describe()

,length
count,5635.000000
mean,93.395386
std,51.075096
min,7.000000
25%,58.000000
50%,85.000000
75%,118.000000
max,561.000000


In [9]:
df_eap=df_train[df_train['author']=='EAP']
df_eap.describe()

,length
count,7900.000000
mean,81.543165
std,60.100183
min,5.000000
25%,40.000000
50%,66.000000
75%,106.000000
max,925.000000


In [10]:
df_mws=df_train[df_train['author']=='MWS']
df_mws.describe()

,length
count,6044.000000
mean,86.124586
std,71.976281
min,4.000000
25%,48.000000
50%,74.000000
75%,108.000000
max,2715.000000


In [11]:
wordset=set()

In [12]:
#делаю сет со всеми словами
for i in df_train.index:
    wordset |= set(df_train['cleaned_text'][i])
wordlist=list(wordset)

In [13]:
#делаю фрейм со словами
df_word=pd.DataFrame(columns=["word", "mws", "eap", "hpl", "all"])
df_word["word"]=wordlist
df_word["mws"]=0
df_word["eap"]=0
df_word["hpl"]=0
df_word["all"]=0
df_word.head()

,word,mws,eap,hpl,all
0,stabl,0,0,0,0
1,barroom,0,0,0,0
2,faulti,0,0,0,0
3,shapeless,0,0,0,0
4,overthrew,0,0,0,0


In [14]:
#Считаем повторения
for i in df_mws.index:
    for word in df_mws['cleaned_text'][i]:
        df_word.loc[df_word["word"]==word,"mws"]+=1
        
for i in df_eap.index:
    for word in df_eap['cleaned_text'][i]:
        df_word.loc[df_word["word"]==word,"eap"]+=1
        
for i in df_hpl.index:
    for word in df_hpl['cleaned_text'][i]:
        df_word.loc[df_word["word"]==word,"hpl"]+=1

KeyboardInterrupt: 

In [ ]:
print(df_word.loc[df_word["word"]=="sai", "mws"])